#  データの準備

In [1]:
import pandas as pd

In [2]:
# データの読み込み
email_data = pd.read_csv('http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')

In [4]:
email_data

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
63995,10,2) $100 - $200,105.54,1,0,Urban,0,Web,Mens E-Mail,0,0,0.0
63996,5,1) $0 - $100,38.91,0,1,Urban,1,Phone,Mens E-Mail,0,0,0.0
63997,6,1) $0 - $100,29.99,1,0,Urban,1,Phone,Mens E-Mail,0,0,0.0
63998,1,5) $500 - $750,552.94,1,0,Surburban,1,Multichannel,Womens E-Mail,0,0,0.0


In [152]:
email_data.describe()

,recency,history,mens,womens,newbie,visit,conversion,spend
count,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000
mean,5.763734,242.085656,0.551031,0.549719,0.502250,0.146781,0.009031,1.050908
std,3.507592,256.158608,0.497393,0.497526,0.499999,0.353890,0.094604,15.036448
min,1.000000,29.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,64.660000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,158.110000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,9.000000,325.657500,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,12.000000,3345.930000,1.000000,1.000000,1.000000,1.000000,1.000000,499.000000


In [69]:
# 調査：CV数は非常に少ない
email_data.groupby('conversion').agg({'spend': ['size', 'min', 'mean', 'max']})

spend                          
             size    min        mean    max
conversion                                 
0           63422   0.00    0.000000    0.0
1             578  29.99  116.363547  499.0

In [33]:
# 女性向けメール配信データの削除と介入変数の追加
male_df = email_data.copy()  # SettingWithCoppyWarningを回避するためにコピーしているが、本質的に意味はない
male_df = male_df[male_df['segment'] != 'Womens E-Mail']
male_df['treatment'] = male_df['segment'].apply(lambda x: 1 if x == 'Mens E-Mail' else 0)

In [130]:
male_df

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0.0,1
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,No E-Mail,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63994,7,1) $0 - $100,86.46,0,1,Urban,0,Web,Mens E-Mail,0,0,0.0,1
63995,10,2) $100 - $200,105.54,1,0,Urban,0,Web,Mens E-Mail,0,0,0.0,1
63996,5,1) $0 - $100,38.91,0,1,Urban,1,Phone,Mens E-Mail,0,0,0.0,1
63997,6,1) $0 - $100,29.99,1,0,Urban,1,Phone,Mens E-Mail,0,0,0.0,1


# RCTの集計と有意差検定

In [137]:
# 集計による比較
# summary_by_segment = male_df.groupby('treatment').agg({'conversion': 'mean', 'spend': 'mean', 'recency': 'size'}, )
# summary_by_segment.columns = ['conversion_rate', 'spend_mean', 'count']
summary_by_segment = (
    male_df
    .groupby('treatment')
    .agg(
        conversion_rate=('conversion', 'mean'),
        spend_mean=('spend', 'mean'),
        count=('recency', 'size'),
    )
)

In [138]:
summary_by_segment

,conversion_rate,spend_mean,count
treatment,,,
0,0.005726,0.652789,21306
1,0.012531,1.422617,21307


In [55]:
# 有意差検定
mens_mail = male_df[male_df['treatment'] == 1]['spend']
no_mail = male_df[male_df['treatment'] == 0]['spend']


In [70]:
from scipy import stats

stats.ttest_ind(mens_mail, no_mail, equal_var=True)

Ttest_indResult(statistic=5.300090294465472, pvalue=1.163200872605869e-07)

# バイアスのあるデータによる効果の検証

In [148]:
# バイアスのあるデータの準備
import numpy as np
np.random.seed(529)

obs_rate_c = 0.5
obs_rate_t = 0.5

biased_data = male_df.copy()
biase_rule = (biased_data['history'] > 300) | (biased_data['recency'] < 6) | (biased_data['channel'] == 'Multichannel')
biased_data['obs_rate_c'] = biase_rule.apply(lambda x: obs_rate_c if x else 1)
biased_data['obs_rate_t'] = biase_rule.apply(lambda x: 1 if x else obs_rate_t)
biased_data['random_number'] = np.random.uniform(size=biased_data.shape[0])
biased_data = (
    biased_data[
                ((biased_data['treatment'] == 0) & (biased_data['random_number'] < biased_data['obs_rate_c']))
                | ((biased_data['treatment'] == 1) & (biased_data['random_number'] < biased_data['obs_rate_t']))
                ]
)

# Rだと変数名と列名で同じobs_rate_c/obs_rate_tを参照してるからややこしい

In [149]:
biased_data

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment,obs_rate_c,obs_rate_t,random_number
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0,0.5,1.0,0.225526
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1,0.5,1.0,0.820611
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1,0.5,1.0,0.948958
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0.0,1,0.5,1.0,0.621804
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,No E-Mail,0,0,0.0,0,0.5,1.0,0.411938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63990,6,1) $0 - $100,80.02,0,1,Surburban,0,Phone,No E-Mail,0,0,0.0,0,1.0,0.5,0.943821
63992,1,5) $500 - $750,519.69,1,1,Urban,1,Phone,Mens E-Mail,0,0,0.0,1,0.5,1.0,0.328752
63995,10,2) $100 - $200,105.54,1,0,Urban,0,Web,Mens E-Mail,0,0,0.0,1,1.0,0.5,0.373351
63996,5,1) $0 - $100,38.91,0,1,Urban,1,Phone,Mens E-Mail,0,0,0.0,1,0.5,1.0,0.167203


In [154]:
# 集計
summary_by_segment_biased = (
    biased_data
    .groupby('treatment')
    .agg(
        conversion_rate=('conversion', 'mean'),
        spend_mean=('spend', 'mean'),
        count=('recency', 'size'),
    )
)

In [155]:
summary_by_segment_biased

,conversion_rate,spend_mean,count
treatment,,,
0,0.004990,0.635578,14830
1,0.013415,1.527637,17220


In [129]:
# 有意差検定
mens_mail_biased = biased_data[biased_data['treatment'] == 1]['spend']
no_mail_biased = biased_data[biased_data['treatment'] == 0]['spend']

stats.ttest_ind(mens_mail, no_mail_biased, equal_var=True)

Ttest_indResult(statistic=4.671796348585924, pvalue=2.9965111266370564e-06)